# Uncertainty of soil water content estimated from epithermal neutrons

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import bokeh

In [2]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show

In [3]:
def get_N(N0=None, thc=None, rhob=None, a0=0.0808, a1=0.372, a2=0.115, rhow=1000.):
    return N0 * ( (a0 / (thc * rhow / rhob + a2)) + a1)

def N_error(N, a=24):
    return (np.sqrt(N*a) / a)
#    return (np.sqrt(N*a) / np.sqrt(a))

In [4]:
def dth_dN(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return 111.111 * a0 * rhob * (mb - 111.111) * fs * N0 / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)

def dth_dN0(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return 111.111 * a0 * rhob * (mb - 111.111) * fs * N / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)

def dth_dfs(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return 111.111 * a0 * rhob * (mb - 111.111) * N * N0 / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)

def dth_dmb(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return 0.009 * a0 * rhob * fs * N * N0 / (rhow * (a1 * N0 * (0.009 * mb - 1) + fs * N)**2)

def dth_drhob(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return ((a0 / (fs * N / (N0 * (1. - 0.009*mb)) - a1)) - a2 - 0.556 * som - lw) / rhow

def dth_dsom(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return -0.556 * rhob / rhow

def dth_dlw(N=None, N0=None, fs=None, rhob=None, mb=None, som=None, lw=None, a0=None, a1=None, a2=None, rhow=None):
    return -rhob / rhow


def sigma_th(pars, sigmas):
    return np.sqrt( 
              dth_dfs(**pars)**2   * sigmas["fs"]**2   + \
              dth_dN(**pars)**2    * sigmas["N"]**2    + \
              dth_dN0(**pars)**2   * sigmas["N0"]**2   + \
              dth_dmb(**pars)**2   * sigmas["mb"]**2   + \
              dth_drhob(**pars)**2 * sigmas["rhob"]**2 + \
              dth_dsom(**pars)**2  * sigmas["som"]**2  + \
              dth_dlw(**pars)**2   * sigmas["lw"]**2
    )

In [5]:
def get_sigma_fs(fs, Ncal, a=24):
    """Error of fs for a given collocation interval a and a calibrator neutron intensity of Ncal
    
    The relative error of the target variable is the difference between 
    the relative error of the enumerator and the relative error of the denominator.
    """
    return fs * np.sqrt((1+fs)/(Ncal*a))
#    return (1 + np.sqrt(fs)) / (np.sqrt(Ncal*a))

In [6]:
def dth_dN_eff(N=None, Nc=None, rhob=None, thc=None, a0=None, a1=None, a2=None, rhow=None):
    return -a0 * rhob * Nc * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a1 * rhob + a1 * thc * rhow) / \
           (rhow * (a2 * a1 * rhob * (Nc - N) - a0 * rhob * N + a1 * thc * rhow * (Nc - N))**2)
# const dth_dN_eff = -a0 * rhob * Nc * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a1 * rhob + a1 * thc * rhow) / (rhow * (a2 * a1 * rhob * (Nc - N) - a0 * rhob * N + a1 * thc * rhow * (Nc - N))**2)

def dth_dNc_eff(N=None, Nc=None, rhob=None, thc=None, a0=None, a1=None, a2=None, rhow=None):
    return a0 * rhob * N * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a2 * rhob + a1 * thc * rhow) / \
           (rhow * (a1 * (N - Nc) * (a2 * rhob + thc * rhow) + a0 * rhob * N)**2 )
# const dth_dNc_eff = a0 * rhob * N * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a2 * rhob + a1 * thc * rhow) / (rhow * (a1 * (N - Nc) * (a2 * rhob + thc * rhow) + a0 * rhob * N)**2 )

def dth_dthc_eff(N=None, Nc=None, rhob=None, thc=None, a0=None, a1=None, a2=None, rhow=None):
    return a0**2 * rhob**2 * N * Nc / ( (a1 * (N - Nc) * (a2 * rhob + rhow * thc) + a0 * rhob * N)**2 )
# const dth_dthc_eff = a0**2 * rhob**2 * N * Nc / ( (a1 * (N - Nc) * (a2 * rhob + rhow * thc) + a0 * rhob * N)**2 )

def dth_drhob_eff(N=None, Nc=None, rhob=None, thc=None, a0=None, a1=None, a2=None, rhow=None):
    return ( (a1 * (N-Nc) * (a2*rhob+thc*rhow) + a0*N*rhob) \
              * (a0 * Nc * (a2*rhob + thc*rhow) - a2 * (a1*(N-Nc)*(a2*rhob + thc*rhow) + a0*N*rhob) ) \
              - a0**2 * Nc * N * thc * rhow * rhob) / \
              (rhow * (a1 * (N-Nc) * (a2*rhob + thc*rhow) + a0*N*rhob)**2 )
# const dth_drhob_eff = ( (a1 * (N-Nc) * (a2*rhob+thc*rhow) + a0*N*rhob) * (a0 * Nc * (a2*rhob + thc*rhow) - a2 * (a1*(N-Nc)*(a2*rhob + thc*rhow) + a0*N*rhob) ) - a0**2 * Nc * N * thc * rhow * rhob) / (rhow * (a1 * (N-Nc) * (a2*rhob + thc*rhow) + a0*N*rhob)**2 )

def sigma_th_eff(pars, sigmas):
    return np.sqrt( 
              dth_dN_eff(**pars)**2    * sigmas["N"]**2   + \
              dth_dNc_eff(**pars)**2   * sigmas["Nc"]**2  + \
              dth_dthc_eff(**pars)**2  * sigmas["thc"]**2 + \
              dth_drhob_eff(**pars)**2 * sigmas["rhob"]**2
    )
#const sigma_th_eff = Math.sqrt(dth_dN_eff**2 * sigma_N**2 + dth_dNc_eff**2 * sigma_Nc**2 + dth_dthc_eff**2 * sigma_thc**2 + dth_drhob_eff**2 * sigma_rhob**2 )

In [7]:
# const dth_dN_eff = -a0 * rhob * Nc * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a1 * rhob + a1 * thc * rhow) / (rhow * (a2 * a1 * rhob * (Nc - N) - a0 * rhob * N + a1 * thc * rhow * (Nc - N))**2)
# const dth_dNc_eff = a0 * rhob * N * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a2 * rhob + a1 * thc * rhow) / (rhow * (a1 * (N - Nc) * (a2 * rhob + thc * rhow) + a0 * rhob * N)**2 )
# const dth_dthc_eff = a0**2 * rhob**2 * N * Nc / ( (a1 * (N - Nc) * (a2 * rhob + rhow * thc) + a0 * rhob * N)**2 )
# const dth_drhob_eff = ( (a1 * (N-Nc) * (a2*rhob+thc*rhow) + a0*N*rhob) * (a0 * Nc * (a2*rhob + thc*rhow) - a2 * (a1*(N-Nc)*(a2*rhob + thc*rhow) + a0*N*rhob) ) - a0**2 * Nc * N * thc * rhow * rhob) / (rhow * (a1 * (N-Nc) * (a2*rhob + thc*rhow) + a0*N*rhob)**2 )
# const sigma_th_eff = Math.sqrt(dth_dN_eff**2 * sigma_N**2 + dth_dNc_eff**2 * sigma_Nc**2 + dth_dthc_eff**2 * sigma_thc**2 + dth_drhob_eff**2 * sigma_rhob**2 )

In [8]:
def get_pars_n_sigmas_local(N0, N1, N2, thc, rhob, fs, sigma_thc, sigma_rhob, fc=0.4):
    pars = {"rhob":rhob, "thc":thc, "a0":0.0808, "a1":0.372, "a2":0.115, "rhow":1000.}
    pars["Nc"] = get_N(N0, thc, rhob)
    pars["N"] = np.linspace(N1,N2,100)
    sigmas = {"Nc":N_error(pars["Nc"]/fs_)*fs_, "thc":sigma_thc, "rhob":sigma_rhob}
    sigmas["N"] = N_error(np.linspace(N1/fs_,N2/fs_,100))*fs_
#    N_at_fc = get_N(N0=avgn0, thc=fc, rhob=rhob_)
    return pars, sigmas#, N_at_fc

def get_pars_n_sigmas_general(N0, N1, N2, rhob, fs, mb, som, lw, sigma_N0, sigma_rhob, sigma_mb, 
                              sigma_som, sigma_lw, fc=0.4):

    pars_general = {"N0":N0, "fs":fs, "rhob":rhob, "mb":mb, "som":som, "lw":lw,
                    "a0":0.0808, "a1":0.372, "a2":0.115, "rhow":1000.}

    sigmas_general = {"N0":sigma_N0, "fs":0., "mb": sigma_mb, "rhob":sigma_rhob, 
                      "som":sigma_som, "lw":sigma_lw}
    
    pars_general["N"] = np.linspace(N1,N2,100)
    sigmas_general["N"] = N_error(pars_general["N"]/fs_)*fs_
    sigma_th_N_general = sigma_th(pars_general, sigmas_general)
    
    return pars_general, sigmas_general

# decoration
def decorate(ax, fc, rhob, ylabel=r"$\sigma_{\theta(N)}$, m³/m³", ymax=0.2):
    N_at_fc=get_N(N0=avgn0, thc=fc, rhob=rhob)
    plt.fill_betweenx(y=[0,ymax], x1=N1*0.8, x2=N_at_fc, color="black", alpha=0.2)
    ax.axvline(N_at_fc, lw=0.5, color="black")
    plt.xlabel(r"N (cph)")
    plt.xlim(N1,N2)
    plt.ylim(0,ymax)
    plt.ylabel(ylabel)
    xtxt = np.mean(plt.xlim())
    ytxt = plt.ylim()[-1]
#     plt.text(xtxt, 1.12*ytxt, r"$\sigma_{\rho_b}$=30 g/cm³, $\sigma_{m_b}$=0.2 kg/m²",fontsize=12, horizontalalignment="center")
#     plt.text(xtxt, 1.05*ytxt, r"$\sigma_{ \theta_{c}}$=0.03 m³/m³",fontsize=12, horizontalalignment="center")
    plt.legend(fontsize=10)
    plt.grid(axis="y")

gcols = "#641e16", "#c0392b", "#d98880"
lcols = "#154360", "#2980b9", "#7fb3d5"

In [9]:
# # 4 scenarios
# # scenario 1: effect of calibration date
# # scenario 2: effect of bulk density error
# # scenario 3: effect of biomass error
# # scenario 4: effect of calibration theta error

# fc=0.4
# avgn0 = 2302


# # ALL
# rhob_ = 1300.
# sigma_rhob = 130
# N1, N2 = 1200, 1700

# # LOCAL
# thcs = [0.15, 0.25, 0.35]
# sigma_thc = 0.025

# # GENERAL
# fs_ = 1.
# sigma_fs = get_sigma_fs(fs_, 1400, 48)
# mb_ = 2
# sigma_mb = 0.2
# som_ = 0.06
# sigma_som = 0.01
# lw_ = 0.02
# sigma_lw = 0.002
# N0 = 2302
# sigma_N0=19
# #sigma_N0 = (n0suci - n0slci) / 2. / 1.96
    
# plt.rc('font', **{'size'   : 12})
# fig, ax = plt.subplots(1,4, figsize=(12,3), sharey=True)
# ax = ax.ravel()
# ymax = 0.2

# # --------------------------------
# # SCENARIO 1
# plt.sca(ax[0])

# for i, thc_ in enumerate(thcs):
#     pars, sigmas = get_pars_n_sigmas_local(N0=avgn0, N1=N1, N2=N2, thc=thc_,
#                                               rhob=rhob_, fs=1., sigma_thc=sigma_thc, sigma_rhob=sigma_rhob)
#     sigma_th_N = sigma_th_eff(pars, sigmas)
    
#     pl = plt.plot(pars["N"], sigma_th_N, color=lcols[i], label=r"$\theta_c$=%.2f" % thc_)
#     plt.plot(pars["Nc"], sigma_thc, "kx", mew=1.5, color=lcols[i], ms=10, label=r"_dummy")


# pars_general, sigmas_general = get_pars_n_sigmas_general(N0=avgn0, N1=N1, N2=N2, rhob=rhob_, fs=fs_, 
#                                                          mb=mb_, som=som_, lw=lw_, sigma_N0=sigma_N0, 
#                                                          sigma_rhob=sigma_rhob, sigma_mb=sigma_mb,
#                                                          sigma_som=sigma_som, sigma_lw=sigma_lw)
# sigma_th_N_general = sigma_th(pars_general, sigmas_general)
# plt.plot(pars_general["N"], sigma_th_N_general, color=gcols[1], label="_dummy")
# decorate(ax[0], fc=fc, rhob=rhob_)

# # --------------------------------
# # SCENARIO 2
# plt.sca(ax[1])

# for i, sigma_rhob_ in enumerate([50, 100, 200]):
#     pars_general, sigmas_general = get_pars_n_sigmas_general(N0=avgn0, N1=N1, N2=N2, rhob=rhob_, fs=fs_, 
#                                                              mb=mb_, som=som_, lw=lw_, sigma_N0=sigma_N0, 
#                                                              sigma_rhob=sigma_rhob_, sigma_mb=sigma_mb,
#                                                              sigma_som=sigma_som, sigma_lw=sigma_lw)
#     sigma_th_N_general = sigma_th(pars_general, sigmas_general)
#     plt.plot(pars_general["N"], sigma_th_N_general, color=gcols[i], label=r"$\sigma_{\rho_b}$=%d$\,$kg/m³" % sigma_rhob_)
    
# pars, sigmas = get_pars_n_sigmas_local(N0=avgn0, N1=N1, N2=N2, thc=0.25,
#                                           rhob=rhob_, fs=1., sigma_thc=sigma_thc, sigma_rhob=sigma_rhob_)
# sigma_th_N = sigma_th_eff(pars, sigmas)
# pl = plt.plot(pars["N"], sigma_th_N, color=lcols[1], label="_dummy")
# plt.plot(pars["Nc"], sigma_thc, "kx", mew=1.5, color=lcols[1], ms=10, label=r"_dummy")

# decorate(ax[1], fc=fc, rhob=rhob_, ylabel="")
# plt.ylim(0,0.2)


# # --------------------------------
# # SCENARIO 3
# plt.sca(ax[2])

# for i, sigma_mb_ in enumerate([0.2, 1, 5]):
#     pars_general, sigmas_general = get_pars_n_sigmas_general(N0=avgn0, N1=N1, N2=N2, rhob=rhob_, fs=fs_, 
#                                                              mb=mb_, som=som_, lw=lw_, sigma_N0=sigma_N0, 
#                                                              sigma_rhob=sigma_rhob, sigma_mb=sigma_mb_,
#                                                              sigma_som=sigma_som, sigma_lw=sigma_lw)
#     sigma_th_N_general = sigma_th(pars_general, sigmas_general)
#     plt.plot(pars_general["N"], sigma_th_N_general, color=gcols[i], label=r"$\sigma_{m_b}$=%.1f$\,$kg/m²" % sigma_mb_)
    
# pars, sigmas = get_pars_n_sigmas_local(N0=avgn0, N1=N1, N2=N2, thc=0.25,
#                                           rhob=rhob_, fs=1., sigma_thc=sigma_thc, sigma_rhob=sigma_rhob_)
# sigma_th_N = sigma_th_eff(pars, sigmas)
# pl = plt.plot(pars["N"], sigma_th_N, color=lcols[1], label="_dummy")
# plt.plot(pars["Nc"], sigma_thc, "kx", mew=1.5, color=lcols[1], ms=10, label="_dummy")

# decorate(ax[2], fc=fc, rhob=rhob_, ylabel="")

# # --------------------------------
# # SCENARIO 4
# plt.sca(ax[3])

# for i, sigma_thc_ in enumerate([0.01, 0.03, 0.05]):
#     pars, sigmas = get_pars_n_sigmas_local(N0=avgn0, N1=N1, N2=N2, thc=0.25,
#                                           rhob=rhob_, fs=1., sigma_thc=sigma_thc_, sigma_rhob=sigma_rhob_)
#     sigma_th_N = sigma_th_eff(pars, sigmas)
#     pl = plt.plot(pars["N"], sigma_th_N, color=lcols[i], label=r"$\sigma_{\theta_c}$=%.2f" % sigma_thc_)
#     plt.plot(pars["Nc"], sigma_thc_, "kx", mew=1.5, color=lcols[i], ms=10, label=r"_dummy")


# pars_general, sigmas_general = get_pars_n_sigmas_general(N0=avgn0, N1=N1, N2=N2, rhob=rhob_, fs=fs_, 
#                                                          mb=mb_, som=som_, lw=lw_, sigma_N0=sigma_N0, 
#                                                          sigma_rhob=sigma_rhob, sigma_mb=sigma_mb,
#                                                          sigma_som=sigma_som, sigma_lw=sigma_lw)
# sigma_th_N_general = sigma_th(pars_general, sigmas_general)
# plt.plot(pars_general["N"], sigma_th_N_general, color=gcols[1], label="_dummy")
    

# decorate(ax[3], fc=fc, rhob=rhob_, ylabel="")



# plt.ylim(0,0.10)

# ax[0].set_title(r"Scenario 1", fontsize=12)
# ax[1].set_title(r"Scenario 2", fontsize=12)
# ax[2].set_title(r"Scenario 3", fontsize=12)
# ax[3].set_title(r"Scenario 4", fontsize=12)

# plt.tight_layout()



In [33]:
fc=0.4
avgn0 = 2302

# ALL
rhob_ = 1300.
sigma_rhob = 130
N1, N2 = 1200, 1900

# LOCAL
thcs = [0.15, 0.25, 0.35]
thc_ = 0.25
sigma_thc = 0.025

# GENERAL
fs_ = 1.
sigma_fs = get_sigma_fs(fs_, 1400, 48)
mb_ = 2
sigma_mb = 0.2
som_ = 0.06
sigma_som = 0.01
lw_ = 0.02
sigma_lw = 0.002
N0 = 2302
sigma_N0=19
#sigma_N0 = (n0suci - n0slci) / 2. / 1.96
    
pars, sigmas = get_pars_n_sigmas_local(N0=avgn0, N1=N1, N2=N2, thc=thc_,
                                              rhob=rhob_, fs=1., sigma_thc=sigma_thc, sigma_rhob=sigma_rhob)
sigma_th_N = sigma_th_eff(pars, sigmas)
pars_general, sigmas_general = get_pars_n_sigmas_general(N0=avgn0, N1=N1, N2=N2, rhob=rhob_, fs=fs_, 
                                                         mb=mb_, som=som_, lw=lw_, sigma_N0=sigma_N0, 
                                                         sigma_rhob=sigma_rhob, sigma_mb=sigma_mb,
                                                         sigma_som=sigma_som, sigma_lw=sigma_lw)
sigma_th_N_general = sigma_th(pars_general, sigmas_general)

In [50]:
sigmas_general

{'N0': 19,
 'fs': 0.0,
 'mb': 0.2,
 'rhob': 130,
 'som': 0.01,
 'lw': 0.002,
 'N': array([7.07106781, 7.09186949, 7.11261032, 7.13329085, 7.1539116 ,
        7.17447308, 7.1949758 , 7.21542026, 7.23580696, 7.25613638,
        7.276409  , 7.2966253 , 7.31678574, 7.33689078, 7.35694088,
        7.37693649, 7.39687804, 7.41676597, 7.43660072, 7.45638271,
        7.47611235, 7.49579006, 7.51541625, 7.53499133, 7.55451567,
        7.57398969, 7.59341377, 7.61278828, 7.63211362, 7.65139014,
        7.67061822, 7.68979822, 7.7089305 , 7.72801541, 7.74705331,
        7.76604454, 7.78498944, 7.80388835, 7.8227416 , 7.84154953,
        7.86031245, 7.87903069, 7.89770456, 7.91633439, 7.93492048,
        7.95346313, 7.97196265, 7.99041935, 8.00883351, 8.02720543,
        8.04553539, 8.06382369, 8.08207061, 8.10027642, 8.11844141,
        8.13656584, 8.15464999, 8.17269413, 8.19069851, 8.2086634 ,
        8.22658907, 8.24447575, 8.26232372, 8.28013322, 8.29790449,
        8.31563778, 8.33333333, 8.

# Bokeh plot

In [64]:
x = pars["N"]
y = sigma_th(pars_general, sigmas_general)
yl = sigma_th_eff(pars, sigmas)

bokeh.io.output_file("docs/interactive.html")

source = ColumnDataSource(data=dict(x=x, y=y, yl=yl))
src_thc = ColumnDataSource(data=dict(Nc=[pars["Nc"]], sigma_thc=[sigmas["thc"]]))

thetas = np.array([0.1, 0.2, 0.3, 0.4])
Ns = get_N(N0, thetas, pars["rhob"])
src_line01 = ColumnDataSource(data=dict(x=[Ns[0], Ns[0]], y=[0,0.5]))
src_line02 = ColumnDataSource(data=dict(x=[Ns[1], Ns[1]], y=[0,0.5]))
src_line03 = ColumnDataSource(data=dict(x=[Ns[2], Ns[2]], y=[0,0.5]))
src_line04 = ColumnDataSource(data=dict(x=[Ns[3], Ns[3]], y=[0,0.5]))

plot = figure(x_range=(1200, 1900),  y_range=(0, 0.12), width=500, height=400, x_axis_label='Neutron intensity (cph)', 
           y_axis_label='Error of SWC (m³/m³)')


gcols = "#641e16", "#c0392b", "#d98880"
lcols = "#154360", "#2980b9", "#7fb3d5"
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6, line_color=gcols[1], legend="general calibration")
plot.line('x', 'yl', source=source, line_width=3, line_alpha=0.6, line_color=lcols[1], legend="local calibration")
plot.circle("Nc", "sigma_thc", source=src_thc, size=12, 
            color=lcols[1], fill_alpha=0., legend="Error of ref. SWC")
plot.line("x", "y", source=src_line01, color="grey")#, legend="0.1 m³/m³")
plot.line("x", "y", source=src_line02, color="grey")#, legend="0.2 m³/m³")
plot.line("x", "y", source=src_line03, color="grey")#, legend="0.3 m³/m³")
plot.line("x", "y", source=src_line04, color="grey")#, legend="0.4 m³/m³")

src_labels = ColumnDataSource(dict(x=Ns, y=[0.09]*4, 
                                   text=["0.1 m³/m³", "0.2 m³/m³", "0.3 m³/m³", "0.4 m³/m³"]))
glyph = bokeh.models.Text(x="x", y="y", text="text", angle=90, angle_units="deg", text_color="grey")
plot.add_glyph(src_labels, glyph)

plot.legend.location = "center_right"
#plot.legend.title= "Calibration strategy"

fsize = "13pt"
plot.xaxis.axis_label_text_font_size = fsize
plot.xaxis.major_label_text_font_size = fsize

plot.yaxis.axis_label_text_font_size = fsize
plot.yaxis.major_label_text_font_size = fsize

plot.legend.label_text_font_size = fsize
plot.legend.title_text_font_size = fsize


sl_sigma_mb = Slider(start=0, end=10, value=sigmas_general["mb"], step=.1, title="Error of AGB (kg/m²)")
sl_sigma_rhob = Slider(start=0, end=400., value=sigmas_general["rhob"], step=1, title="Error of bulk density (kg/m³)")
sl_sigma_som = Slider(start=0.0, end=0.08, value=sigmas_general["som"], step=.001, title="Error of SOM (g/g)")

sl_thc = Slider(start=0.1, end=0.4, value=pars["thc"], step=.001, title=r"SWC (m³/m³)")
sl_sigma_thc = Slider(start=0.0, end=0.1, value=sigmas["thc"], step=.001, title=r"Error of reference SWC (m³/m³)")

sl_rhob = Slider(start=800, end=1600, value=pars["rhob"], step=1, title="Bulk density (kg/m³)")

# GLOBAL
# const dth_dN = 111.111 * a0 * rhob * (mb - 111.111) * fs * N0 / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)
# const dth_dN0 = 111.111 * a0 * rhob * (mb - 111.111) * fs * N / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)
# const dth_dfs = 111.111 * a0 * rhob * (mb - 111.111) * N * N0 / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)
# const dth_dmb = 0.009 * a0 * rhob * fs * N * N0 / (rhow * (a1 * N0 * (0.009 * mb - 1) + fs * N)**2)
# const = ((a0 / (fs * N / (N0 * (1. - 0.009*mb)) - a1)) - a2 - 0.556 * som - lw) / rhow
# const dth_dsom = -0.556 * rhob / rhow
# const dth_dlw = -rhob / rhow
# const sigma_th sqrt = dth_dfs**2 * sigma_fs**2 + dth_dN**2 * sigma_N**2 + dth_dN0**2 * sigma_N0**2 + dth_dmb**2 * sigma_mb**2 + dth_drhob**2 * sigma_rhob**2 + dth_dsom**2 * sigma_som**2 + dth_dlw**2 * sigma_lw**2

# LOCAL
# const dth_dN_eff = -a0 * rhob * Nc * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a1 * rhob + a1 * thc * rhow) / (rhow * (a2 * a1 * rhob * (Nc - N) - a0 * rhob * N + a1 * thc * rhow * (Nc - N))**2)
# const dth_dNc_eff = a0 * rhob * N * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a2 * rhob + a1 * thc * rhow) / (rhow * (a1 * (N - Nc) * (a2 * rhob + thc * rhow) + a0 * rhob * N)**2 )
# const dth_dthc_eff = a0**2 * rhob**2 * N * Nc / ( (a1 * (N - Nc) * (a2 * rhob + rhow * thc) + a0 * rhob * N)**2 )
# const dth_drhob_eff = ( (a1 * (N-Nc) * (a2*rhob+thc*rhow) + a0*N*rhob) * (a0 * Nc * (a2*rhob + thc*rhow) - a2 * (a1*(N-Nc)*(a2*rhob + thc*rhow) + a0*N*rhob) ) - a0**2 * Nc * N * thc * rhow * rhob) / (rhow * (a1 * (N-Nc) * (a2*rhob + thc*rhow) + a0*N*rhob)**2 )
# const sigma_th_eff = Math.sqrt(dth_dN_eff**2 * sigma_N**2 + dth_dNc_eff**2 * sigma_Nc**2 + dth_dthc_eff**2 * sigma_thc**2 + dth_drhob_eff**2 * sigma_rhob**2 )

callback = CustomJS(args=dict(source=source,
                              src_thc=src_thc,
                              src_line01=src_line01,
                              src_line02=src_line02,
                              src_line03=src_line03,
                              src_line04=src_line04,
                              src_labels=src_labels,
                              sl_sigma_mb=sl_sigma_mb, 
                              sl_sigma_rhob=sl_sigma_rhob,
                              sl_sigma_som=sl_sigma_som,
                              sl_thc=sl_thc,
                              sl_sigma_thc=sl_sigma_thc,
                              sl_rhob=sl_rhob),
                    code="""
    const data = source.data;

    const x = data['x']
    const y = data['y']
    const yl = data['yl']
    
    const Nc2 = src_thc.data['Nc']
    const sigma_thc2 = src_thc.data['sigma_thc']
           
    const N0 = 2302
    const fs = 1.
    const rhob = sl_rhob.value
    const mb = 2
    const som = 0.06
    const lw = 0.02
    const a0 = 0.0808
    const a1 = 0.372
    const a2 = 0.115
    const rhow = 1000.
    const sigma_fs = 0.
    const sigma_N0 = 19
    const sigma_mb = sl_sigma_mb.value
    const sigma_rhob = sl_sigma_rhob.value
    const sigma_som = sl_sigma_som.value
    const sigma_lw = 0.002

    
    var N = 0
    var sigma_N = 0
    var dth_dN = 0
    var dth_dN0 = 0
    var dth_dfs = 0
    var dth_dmb = 0
    var dth_drhob = 0
    var dth_dsom = 0
    var dth_dlw = 0
    var sigma_th = 0
    var dth_dN_eff = 0
    var dth_dNc_eff = 0
    var dth_dthc_eff = 0
    var dth_drhob_eff = 0
    var sigma_th_eff = 0
    
    const thc = sl_thc.value
    const sigma_thc = sl_sigma_thc.value
    sigma_thc2[0] = sigma_thc 
    
    var Nc = 0
    var sigma_Nc = 0
    
    const Nlabel = src_labels.data["x"]
    
    const N_at_10 = src_line01.data["x"]
    var tmp = N0 * ( (a0 / (0.1 * rhow / rhob + a2)) + a1)
    N_at_10[0] = tmp 
    N_at_10[1] = tmp
    Nlabel[0] = tmp
    
    const N_at_20 = src_line02.data["x"]
    tmp = N0 * ( (a0 / (0.2 * rhow / rhob + a2)) + a1)
    N_at_20[0] = tmp 
    N_at_20[1] = tmp
    Nlabel[1] = tmp
    
    const N_at_30 = src_line03.data["x"]
    tmp = N0 * ( (a0 / (0.3 * rhow / rhob + a2)) + a1)
    N_at_30[0] = tmp 
    N_at_30[1] = tmp
    Nlabel[2] = tmp
    
    const N_at_40 = src_line04.data["x"]
    tmp = N0 * ( (a0 / (0.4 * rhow / rhob + a2)) + a1)
    N_at_40[0] = tmp 
    N_at_40[1] = tmp
    Nlabel[3] = tmp

    
    for (let i = 0; i < x.length; i++) {
        N = x[i]
        sigma_N = Math.sqrt(N*24) / 24
        //GLOBAL
        dth_dN = 111.111 * a0 * rhob * (mb - 111.111) * fs * N0 / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)
        dth_dN0 = 111.111 * a0 * rhob * (mb - 111.111) * fs * N / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)
        dth_dfs = 111.111 * a0 * rhob * (mb - 111.111) * N * N0 / (rhow * (a1 * (mb-111.111) * N0 + 111.111 * fs * N )**2)
        dth_dmb = 0.009 * a0 * rhob * fs * N * N0 / (rhow * (a1 * N0 * (0.009 * mb - 1) + fs * N)**2)
        dth_drhob = ((a0 / (fs * N / (N0 * (1. - 0.009*mb)) - a1)) - a2 - 0.556 * som - lw) / rhow
        dth_dsom = -0.556 * rhob / rhow
        dth_dlw = -rhob / rhow
        sigma_th = Math.sqrt(dth_dfs**2 * sigma_fs**2 + dth_dN**2 * sigma_N**2 + dth_dN0**2 * sigma_N0**2 + dth_dmb**2 * sigma_mb**2 + dth_drhob**2 * sigma_rhob**2 + dth_dsom**2 * sigma_som**2 + dth_dlw**2 * sigma_lw**2)
        y[i] = sigma_th
        
        //LOCAL
        Nc = N0 * ( (a0 / (thc * rhow / rhob + a2)) + a1)
        Nc2[0] = Nc
        sigma_Nc = Math.sqrt(Nc*24) / 24
        dth_dN_eff = -a0 * rhob * Nc * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a1 * rhob + a1 * thc * rhow) / (rhow * (a2 * a1 * rhob * (Nc - N) - a0 * rhob * N + a1 * thc * rhow * (Nc - N))**2)
        dth_dNc_eff = a0 * rhob * N * (a2 * rhob + thc * rhow) * (a0 * rhob + a2 * a2 * rhob + a1 * thc * rhow) / (rhow * (a1 * (N - Nc) * (a2 * rhob + thc * rhow) + a0 * rhob * N)**2 )
        dth_dthc_eff = a0**2 * rhob**2 * N * Nc / ( (a1 * (N - Nc) * (a2 * rhob + rhow * thc) + a0 * rhob * N)**2 )
        dth_drhob_eff = ( (a1 * (N-Nc) * (a2*rhob+thc*rhow) + a0*N*rhob) * (a0 * Nc * (a2*rhob + thc*rhow) - a2 * (a1*(N-Nc)*(a2*rhob + thc*rhow) + a0*N*rhob) ) - a0**2 * Nc * N * thc * rhow * rhob) / (rhow * (a1 * (N-Nc) * (a2*rhob + thc*rhow) + a0*N*rhob)**2 )
        sigma_th_eff = Math.sqrt(dth_dN_eff**2 * sigma_N**2 + dth_dNc_eff**2 * sigma_Nc**2 + dth_dthc_eff**2 * sigma_thc**2 + dth_drhob_eff**2 * sigma_rhob**2 )
        yl[i] = sigma_th_eff
    }
    source.change.emit();
    src_thc.change.emit();
    src_line01.change.emit();
    src_line02.change.emit();
    src_line03.change.emit();
    src_line04.change.emit();
    src_labels.change.emit();
""")

sl_sigma_mb.js_on_change('value', callback)
sl_sigma_rhob.js_on_change('value', callback)
sl_sigma_som.js_on_change('value', callback)
sl_thc.js_on_change('value', callback)
sl_sigma_thc.js_on_change('value', callback)
sl_rhob.js_on_change('value', callback)

show(row(plot, column(sl_sigma_mb, sl_sigma_rhob, sl_sigma_som, sl_thc, sl_sigma_thc, sl_rhob)))